# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb",
        "src/statics/populations/cbs_population.ipynb"
    ]
}
```

# **TABLES**

---

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTATIC_CBS_POPULATION_AGEGROUP_10]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTATIC_CBS_POPULATION_AGEGROUP_10]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_POPULATION_AGEGROUP_10]'))
CREATE TABLE [VWSSTATIC].[CBS_POPULATION_AGEGROUP_10](
	[ID] INT PRIMARY KEY NONCLUSTERED NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSSTATIC_CBS_POPULATION_AGEGROUP_10]),
  	[AGEGROUP] VARCHAR(10) NULL,
  	[DATUM_PEILING] DATE NULL,
  	[POPULATIE] INT NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 3) CREATE STAGE INDEXE(S).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'CI_DLI_VWSSTATIC_CBS_POPULATION_AGEGROUP_10')
CREATE CLUSTERED INDEX [CI_DLI_VWSSTATIC_CBS_POPULATION_AGEGROUP_10] ON [VWSSTATIC].[CBS_POPULATION_AGEGROUP_10] (
	[DATE_LAST_INSERTED]
);
GO

# **STORED PROCEDURES**

---

### STATIC &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_CBS_POPULATION_AGEGROUP_10]
AS
BEGIN
    WITH BASE_CTE AS (
SELECT 
     DATUM_PEILING
    ,LEEFTIJD
    ,CASE 
        WHEN LEEFTIJD >= 0      AND LEEFTIJD <= 9   THEN '0-9'
        WHEN LEEFTIJD >= 10     AND LEEFTIJD <= 19  THEN '10-19'
        WHEN LEEFTIJD >= 20     AND LEEFTIJD <= 29  THEN '20-29'
        WHEN LEEFTIJD >= 30     AND LEEFTIJD <= 39  THEN '30-39'
        WHEN LEEFTIJD >= 40     AND LEEFTIJD <= 49  THEN '40-49'
        WHEN LEEFTIJD >= 50     AND LEEFTIJD <= 59  THEN '50-59'
        WHEN LEEFTIJD >= 60     AND LEEFTIJD <= 69  THEN '60-69'
        WHEN LEEFTIJD >= 70     AND LEEFTIJD <= 79  THEN '70-79'
        WHEN LEEFTIJD >= 80     AND LEEFTIJD <= 89  THEN '80-89'
        WHEN LEEFTIJD >= 90                         THEN '90+'
        ELSE NULL
        END AS AGEGROUP
    ,POPULATIE
    FROM [VWSSTATIC].[CBS_POPULATION_BASE]
    WHERE [DATE_LAST_INSERTED] = ( SELECT MAX([DATE_LAST_INSERTED])  FROM [VWSSTATIC].[CBS_POPULATION_BASE] )
)
    INSERT INTO VWSSTATIC.CBS_POPULATION_AGEGROUP_10
    (
     [DATUM_PEILING]
    ,[AGEGROUP]
    ,[POPULATIE]
    )
SELECT 
     DATUM_PEILING
    ,AGEGROUP
    ,SUM(POPULATIE) AS POPULATIE
FROM BASE_CTE
WHERE AGEGROUP IS NOT NULL
GROUP BY AGEGROUP, DATUM_PEILING
ORDER BY DATUM_PEILING, AGEGROUP

END;
GO

# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'STATIC_DATA',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

-- 2) UPSERT WORKFLOW(S).....
SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @SCHEDULE = '0 13 * */3 MON-FRI', -- AT 12:00 PM, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO STATIC......
SET @source = 'DBO.SP_CBS_POPULATION_AGEGROUP_10';
SET @source_name = 'SOURCE_CBS_POPULATION_AGEGROUP_10';
SET @source_description = 'MAP: CBS_POPULATION_AGEGROUP_10 FROM STATIC TO STATIC';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO STATIC......
SET @process_name = 'PROCESS_CBS_POPULATION_AGEGROUP_10';
SET @process_description = 'MAP: CBS_POPULATION_AGEGROUP_10 FROM STAGE TO STATIC';
SET @process_source_name = 'SOURCE_CBS_POPULATION_AGEGROUP_10';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_PROCESS_CBS_POPULATION_AGEGROUP_10';
SET @dependency_dataflow_name = 'PROCESS_CBS_POPULATION_AGEGROUP_10';
SET @dependency_process_name = 'PROCESS_STATIC_POPULATION_BASE';
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO